<a href="https://colab.research.google.com/github/Jamess200/BirdnetProject/blob/main/Files/scripts/merge_csv_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount Google Drive
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Clone Repo
import os

# Define path in Google Drive where you want to clone the repository
repo_path = '/content/drive/MyDrive/'

# Check if directory already exists
if not os.path.exists(repo_path):
    os.makedirs(repo_path)

# Change working directory to defined path
os.chdir(repo_path)

# Clone the repository if doesnt exist, otherwise pull latest changes
if not os.path.exists(os.path.join(repo_path, 'MyBirdNetTest')):
    !git clone https://github.com/Jamess200/MyBirdNetTest.git
else:
    os.chdir('MyBirdNetTest')
    !git pull

# Verify cloned repository
!ls

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 5), reused 8 (delta 3), pack-reused 0
Unpacking objects: 100% (12/12), 3.33 MiB | 857.00 KiB/s, done.
From https://github.com/Jamess200/MyBirdNetTest
   05d294c..5e85fa1  main       -> origin/main
Updating 05d294c..5e85fa1
error: Your local changes to the following files would be overwritten by merge:
	Files/data/CSV_data/transect_combined.csv
Please commit your changes or stash them before you merge.
Aborting
Files  README.md


In [ ]:
#@title Change Working Directory
# Change working directory to defined path
os.chdir('/content/drive/MyDrive/MyBirdNetTest/Files/data/CSV_data')

# Verify working directory
print(os.getcwd())

/content/drive/MyDrive/MyBirdNetTest/Files/data/CSV_data


In [ ]:
#@title Create Merged Dataframe File For All Transects
import pandas as pd

# Read in the CSV files
df5 = pd.read_csv('transect_5.csv')
df6 = pd.read_csv('transect_6.csv')
df7 = pd.read_csv('transect_7.csv')
df8 = pd.read_csv('transect_8.csv')

# Add a column to each dataframe to indicate the transect source
df5['transect'] = 5
df6['transect'] = 6
df7['transect'] = 7
df8['transect'] = 8

# Function to extract date and hour from the filename
def extract_datetime(filename):
    date_str = filename.split('_')[1]
    time_str = filename.split('_')[2].split('.')[0]
    datetime_str = date_str + ' ' + time_str
    datetime = pd.to_datetime(datetime_str, format='%Y%m%d %H%M%S')
    return datetime

# Apply the function to create a new datetime column
df5['datetime'] = df5['filename'].apply(extract_datetime)
df6['datetime'] = df6['filename'].apply(extract_datetime)
df7['datetime'] = df7['filename'].apply(extract_datetime)
df8['datetime'] = df8['filename'].apply(extract_datetime)

# Function to convert seconds to hour:minutes:seconds format based on the datetime
def seconds_to_hms(datetime, seconds):
    new_time = datetime + pd.to_timedelta(seconds, unit='s')
    return new_time.strftime('%-H:%M:%S')

# Apply the function to convert start_time and end_time
df5['start_time'] = df5.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df5['end_time'] = df5.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df6['start_time'] = df6.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df6['end_time'] = df6.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df7['start_time'] = df7.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df7['end_time'] = df7.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df8['start_time'] = df8.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df8['end_time'] = df8.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)

# Concatenate the dataframes
concatenated_df = pd.concat([df5, df6, df7, df8], ignore_index=True)

# Extract date and start hour
concatenated_df['date'] = concatenated_df['datetime'].dt.date
concatenated_df['start_hour'] = concatenated_df['datetime'].dt.hour

# Sort the dataframe by transect, date, start_time, common_name, and confidence
concatenated_df = concatenated_df.sort_values(by=['transect', 'date', 'start_time', 'common_name', 'confidence'])

# Reorder the columns to have transect, date, start_hour, time, and readable start/end times
reordered_columns = ['transect', 'date', 'start_hour', 'start_time', 'end_time', 'common_name', 'scientific_name', 'confidence', 'label', 'filename']
concatenated_df = concatenated_df[reordered_columns]

# Display the first few rows of the concatenated dataframe
print("\nNew Dataframe:")
print(concatenated_df.head())
print(concatenated_df.tail())



New Dataframe:
   transect        date  start_hour start_time end_time common_name  \
0         5  2024-05-02           4    4:00:21  4:00:24    Barn Owl   
1         5  2024-05-02           4    4:00:42  4:00:45     Mallard   
2         5  2024-05-02           4    4:00:48  4:00:51     Mallard   
3         5  2024-05-02           4    4:00:51  4:00:54     Mallard   
4         5  2024-05-02           4    4:01:36  4:01:39     Mallard   

      scientific_name  confidence                       label  \
0           Tyto alba    0.259368          Tyto alba_Barn Owl   
1  Anas platyrhynchos    0.612938  Anas platyrhynchos_Mallard   
2  Anas platyrhynchos    0.509237  Anas platyrhynchos_Mallard   
3  Anas platyrhynchos    0.727033  Anas platyrhynchos_Mallard   
4  Anas platyrhynchos    0.262130  Anas platyrhynchos_Mallard   

                  filename  
0  ED3_20240502_040000.wav  
1  ED3_20240502_040000.wav  
2  ED3_20240502_040000.wav  
3  ED3_20240502_040000.wav  
4  ED3_20240502_04000

In [ ]:
#@title Save data as a CSV File
# Save the concatenated dataframe to a CSV file
output_filename = 'transect_combined.csv'
concatenated_df.to_csv(output_filename, index=False)
print(f"The combined dataframe has been saved to {output_filename}")

The combined dataframe has been saved to transect_combined.csv


In [ ]:
#@title Save Data as a Excel File
# Save the dataframe to an Excel file
output_file = 'Bird_Detection_Data.xlsx'
concatenated_df.to_excel(output_file, index=False)
print(f"\nDataframe has been saved to {output_file}")


Dataframe has been saved to Bird_Detection_Data.xlsx
